In [1]:
import numpy as np
import pandas as pd
import json
import urllib
import string
from bs4 import BeautifulSoup
#import seaborn as sb                                      # JR: What does the Seaborn library do? Also, needed to pip install it above
#from itertools import product                             # JR: Don't know this library either
#from matplotlib.collections import LineCollection
#import statsmodels
#import statsmodels.formula.api as smf
#from scipy.stats import chisquare
#from IPython.display import Image
print('Done')

Done


In [2]:
movies = []
colnames = ['Release Date','Movie','Genre','Production Budget','Domestic Box Office to Date','Trailer']
nColumns = len(colnames)
starting_letters = string.printable[0:36]+string.printable[69]
#starting_letters = ['a']
for thisletter in starting_letters:
    thisurl = 'http://www.the-numbers.com/movies/letter/'+thisletter
    print(thisurl)
    thispage = urllib.request.urlopen(thisurl)
    soup = BeautifulSoup(thispage,'html.parser',from_encoding='utf-8')
    firstTable = soup.find('table')
    theseRows = soup.findAll('tr')
    for row in theseRows:
        thisrow = []
        cells = row.findAll('td')
        for cell in cells:
            if (cell.get_text() == u'\xa0'):
                thisrow.append(''.encode('utf-8'))
            else:
                thisrow.append(cell.get_text().replace(u'\xa0',' ').encode('utf-8'))
        if (len(thisrow) == 6):
            movies.append(thisrow)
movies_decoded = [[s.decode('utf-8') for s in t] for t in movies]
print('Done!')

http://www.the-numbers.com/movies/letter/0
http://www.the-numbers.com/movies/letter/1
http://www.the-numbers.com/movies/letter/2
http://www.the-numbers.com/movies/letter/3
http://www.the-numbers.com/movies/letter/4
http://www.the-numbers.com/movies/letter/5
http://www.the-numbers.com/movies/letter/6
http://www.the-numbers.com/movies/letter/7
http://www.the-numbers.com/movies/letter/8
http://www.the-numbers.com/movies/letter/9
http://www.the-numbers.com/movies/letter/a
http://www.the-numbers.com/movies/letter/b
http://www.the-numbers.com/movies/letter/c
http://www.the-numbers.com/movies/letter/d
http://www.the-numbers.com/movies/letter/e
http://www.the-numbers.com/movies/letter/f
http://www.the-numbers.com/movies/letter/g
http://www.the-numbers.com/movies/letter/h
http://www.the-numbers.com/movies/letter/i
http://www.the-numbers.com/movies/letter/j
http://www.the-numbers.com/movies/letter/k
http://www.the-numbers.com/movies/letter/l
http://www.the-numbers.com/movies/letter/m
http://www.

In [8]:
thenumbers = pd.DataFrame.from_dict(movies_decoded)
thenumbers.columns = ['releasedate','title','genre','budget','revenue','trailer']

thenumbers['releasedate'] = pd.to_datetime(thenumbers['releasedate'],format=u"%b\xa0%d,\xa0%Y")#,coerce=True)

thenumbers = thenumbers.replace([u'',u'nan'],[np.nan,np.nan])

thenumbers['budget'] = thenumbers['budget'].dropna().str.replace('$','').apply(lambda x:int(x.replace(',','')))
thenumbers['budget'] = thenumbers['budget'].dropna().apply(lambda x: '${:,.0f}'.format(x))

thenumbers['revenue'] = thenumbers['revenue'].dropna().str.replace('$','').apply(lambda x:int(x.replace(',','')))
thenumbers['revenue'] = thenumbers['revenue'].dropna().apply(lambda x: '${:,.0f}'.format(x))

thenumbers['year'] = thenumbers['releasedate'].apply(lambda x:x.year)

thenumbers['has_trailer'] = thenumbers['trailer'].apply(lambda x: True if (x == 'Play') else False)

thenumbers = thenumbers.drop('trailer',axis=1)

thenumbers.index.name = 'id'

thenumbers.columns = ['releasedate_thenumbers','title_thenumbers','genre_thenumbers','budget_thenumbers','revenue_thenumbers','year_thenumbers','has_trailer_thenumbers']
thenumbers.head(30)

thenumbers.to_csv('revenue.csv',encoding='utf8')
print(thenumbers.shape)
print('Done')

(28847, 7)
Done
